# Market Data Integration Guide

## Overview
This guide covers integrating with TastyTrade's market data API using Python WebSocket implementation, with a focus on asyncio-based approach.

## API Resources
- [Developer Portal](https://developer.tastytrade.com/)
- [Streaming Market Data Documentation](https://developer.tastytrade.com/streaming-market-data/)
- [DXLink Protocol Reference](https://demo.dxfeed.com/dxlink-ws/debug/#/protocol)

## References
- dxFeed API [doc](https://dxfeed.readthedocs.io/en/stable/) | [github](https://github.com/dxFeed/dxfeed-python-api) | [pypi](https://pypi.org/project/dxfeed/)
- Web Widget Demo [docs](https://tools.dxfeed.com/python-demo/?_ga=2.236532102.669062738.1733058631-563197392.1733058631) | [github](https://github.com/dxFeed/dxfeed-python-api-web-widget-example?tab=readme-ov-file)

## WebSocket Implementation Details

### Selected Library
We'll be using the [websockets](https://websockets.readthedocs.io/en/stable/) library for Python, which provides high-level WebSocket implementation (not to be confused with the lower-level `websocket` library).

### Key Resources
- **Documentation**: Complete guides available on [readthedocs.io](https://websockets.readthedocs.io/en/stable/)
- **Reference Implementation**: [Example using Threading](https://github.com/LordKaT/tastytrade_api_thing/blob/main/lib/TTWebsocket.py) by [LordKaT](https://github.com/LordKaT)

### Technical Approach
The implementation will use `asyncio` due to its comprehensive [feature support](https://websockets.readthedocs.io/en/stable/reference/features.html) in the websockets library.

### AsyncIO Learning Resources
For better understanding of the asyncio implementation:
- [How does async/await work in Python 3.5](https://snarky.ca/how-the-heck-does-async-await-work-in-python-3-5/)
- [Cooperative multitasking with Coroutines](https://pymotw.com/3/asyncio/coroutines.html)
- [A Curious course on Coroutines and Concurrency](http://www.dabeaz.com/coroutines/)


In [ ]:
from tastytrade.logging import setup_logging
import logging
import asyncio
from tastytrade.sessions import Credentials
from tastytrade.sessions.requests import AsyncSessionHandler
from tastytrade.sessions.sockets import WebSocketManager
from tastytrade.sessions.dxlink import DXLinkClient
from decimal import Decimal
from datetime import datetime, timedelta

import polars as pl

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

shutdown = asyncio.Event()


setup_logging(
    level=logging.INFO,
    # log_dir="/var/log/tastytrade",
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)


def get_trade_day() -> str:
    trade_day = datetime.now()
    while trade_day.weekday() >= 5:
        trade_day += timedelta(days=1)

    return trade_day.strftime("%y%m%d")


def round_decimal(value, precision=5) -> float:
    target = str(precision)

    if not isinstance(value, Decimal):
        value = Decimal(value)

    if precision > 0:
        result = (value / Decimal(target)).quantize(
            Decimal("1"), rounding="ROUND_HALF_UP"
        ) * Decimal(target)
        return float(result)

    else:
        return float(round(value, abs(precision)))

## Test WebSocket context manager

In [ ]:
"""
TODO 

  1. Find nearest ATM strike
  2. list of symbols
      +-10 pts from ATM strike
      both Calls & Puts
"""

session = await AsyncSessionHandler.create(Credentials(env=ENV))

yy_mm_dd = get_trade_day()

symbols = [
    "SPX",
    "NVDA",
    "BTC/USD:CXTALP",
    "BCH/USD:CXTALP",
    "ETH/USD:CXTALP",
    f".SPXW{yy_mm_dd}C5915",
    f".SPXW{yy_mm_dd}C5910",
    f".SPXW{yy_mm_dd}P5910",
    f".SPXW{yy_mm_dd}P5905",
]

async with WebSocketManager(session) as websocket:
    dxlink_client = DXLinkClient(websocket)
    await dxlink_client.setup_feeds()
    await dxlink_client.subscribe_to_feeds(symbols)

    if TEST:
        await asyncio.sleep(DURATION)
    else:
        await websocket.send_keepalives()

    await dxlink_client.queue_manager.cleanup()

await session.close()

In [6]:
trades = dxlink_client.queue_manager.handlers["Trades"].processors["feed"].df

spx_trades = trades.filter(pl.col("eventSymbol") == "SPX")

spx_price = spx_trades.sort("timestamp")[-1].select("price").item()

In [ ]:
dxlink_client.queue_manager.handlers["Quotes"].processors["feed"].df

In [ ]:
dxlink_client.queue_manager.handlers["Greeks"].processors["feed"].df

In [ ]:
dxlink_client.queue_manager.handlers["Summary"].processors["feed"].df

In [ ]:
dxlink_client.queue_manager.handlers["Profile"].processors["feed"].df

In [ ]:
import sys

sys.exit("Done")

## Test individual components

In [ ]:
session = await AsyncSessionHandler.create(Credentials(env=ENV))

exlink = WebSocketManager(session)
await exlink.open()

In [ ]:
dxlink_client = DXLinkClient(exlink)
await dxlink_client.setup_feeds()
await dxlink_client.subscribe_to_feeds(["SPX"])

In [ ]:
asyncio.all_tasks()

In [ ]:
await exlink.close()

In [ ]:
dxlink_client.session.close()

In [ ]:
dxlink_client

In [16]:
symbol = "SPX"

response = session.session.get(session.base_url + "/option-chains/" + symbol)

In [ ]:
from tastytrade.markets.instruments import get_option_chains
import polars as pl

# Using your existing session
df = await get_option_chains(session, "SPX")
print(df)

In [ ]:
df = pl.DataFrame(response["data"]["items"])
df.head()

In [5]:
from enum import Enum


class OptionType(Enum):
    CALL = "C"
    PUT = "P"


date = get_trade_day()
trade_date = f"20{date[:2]}-{date[2:4]}-{date[4:]}"

In [ ]:
(
    df.with_columns(pl.col("strike-price").cast(pl.Float32(), strict=False))
    .with_columns(
        pl.col("option-type").map_elements(lambda x: OptionType(x).name, return_dtype=str)
    )
    .filter((pl.col("expiration-date") == trade_date))
    .filter((pl.col("strike-price") >= 5700))
    .select(["strike-price", "option-type", "streamer-symbol"])
    .sort("strike-price")
    .head(10)
)